# Big Query Tutorial 

First you need to go to 
https://cloud.google.com/docs/authentication/getting-started to set up an account and an API key. Follow the instructions and then 
    replace the link below with the JSON file that gets downloaded. 

In [4]:
import pandas as pd
import os

pd.set_option('display.max_columns', 500)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/dgoldman/Downloads/Introduction to Hadoop-4afff6dd2a8f.json"

## Compute time bewteen pickup and dropoff.

In [5]:
from google.cloud import bigquery
client = bigquery.Client()


query = (
    """SELECT *,
    TIMESTAMP_DIFF( dropoff_datetime, pickup_datetime, SECOND) as travel_time,
    EXTRACT (DATE FROM pickup_datetime) as date_of_year
    FROM `nyc-tlc.yellow.trips`
    LIMIT 10"""
)
df=pd.io.gbq.read_gbq(query,dialect='standard')


ModuleNotFoundError: No module named 'google'

Here we join with the publicly available weather data. This isn't required, but the goal here is just to show you what's posisble
with big query. Spend some time exploring the public data and coming up with hypotheses!


In [24]:
query = (
    """
    SELECT * FROM 
    (
    SELECT *,
    TIMESTAMP_DIFF( dropoff_datetime, pickup_datetime, SECOND) as travel_time,
     EXTRACT (DATE FROM pickup_datetime) as date_of_year,
     EXTRACT (DAY FROM pickup_datetime) as day_of_year,
     EXTRACT (MONTH FROM pickup_datetime) as month_of_year,
     EXTRACT (YEAR FROM pickup_datetime) as year_of_year
    FROM `nyc-tlc.yellow.trips` ) a 
    LEFT JOIN
    (
     select concat(year,'-',mo,'-',da) as date_of_year2,year,mo,da,temp,visib,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder
     from `bigquery-public-data.noaa_gsod.gsod2015` where stn='725053'
    ) weather_data
    on CAST(a.day_of_year AS STRING)=weather_data.da WHERE CAST(year_of_year AS STRING)='2015'
    LIMIT 100"""
)
df=pd.io.gbq.read_gbq(query,dialect='standard')


In [25]:
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,rate_code,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,imp_surcharge,tip_amount,tolls_amount,total_amount,store_and_fwd_flag,travel_time,date_of_year,day_of_year,month_of_year,year_of_year,date_of_year2,year,mo,da,temp,visib,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder
0,CMT,2015-06-20 04:35:22+00:00,2015-06-20 04:35:22+00:00,-74.005455,40.740761,0.0,0.0,5,1,0.0,CSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,2015-06-20,20,6,2015,2015-11-20,2015,11,20,58.1,8.3,5.5,22.9,64.0,51.1,1.25,999.9,0,1,0,0,0
1,CMT,2015-06-20 04:35:22+00:00,2015-06-20 04:35:22+00:00,-74.005455,40.740761,0.0,0.0,5,1,0.0,CSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,2015-06-20,20,6,2015,2015-12-20,2015,12,20,38.6,10.0,5.8,21.0,43.0,34.0,0.00,999.9,0,0,0,0,0
2,CMT,2015-06-20 04:35:22+00:00,2015-06-20 04:35:22+00:00,-74.005455,40.740761,0.0,0.0,5,1,0.0,CSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,2015-06-20,20,6,2015,2015-04-20,2015,04,20,51.1,5.6,7.2,22.0,64.0,46.0,0.02,999.9,1,1,0,0,0
3,CMT,2015-06-20 04:35:22+00:00,2015-06-20 04:35:22+00:00,-74.005455,40.740761,0.0,0.0,5,1,0.0,CSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,2015-06-20,20,6,2015,2015-05-20,2015,05,20,62.0,9.7,5.8,22.0,73.9,57.0,0.00,999.9,0,0,0,0,0
4,CMT,2015-06-20 04:35:22+00:00,2015-06-20 04:35:22+00:00,-74.005455,40.740761,0.0,0.0,5,1,0.0,CSH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,2015-06-20,20,6,2015,2015-08-20,2015,08,20,80.5,10.0,4.2,22.0,87.1,77.0,0.00,999.9,0,0,0,0,0


# Build a model

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
y = df['travel_time']
X = df[['temp','wdsp']]
X=X.fillna(-1)
rf_clf = RandomForestClassifier(n_estimators=100,
                             random_state=0, max_depth=20)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)


rf_clf.fit(X_train,y_train)
rf_clf.score(X_test,y_test)

0.1